In [1]:
import pandas as pd
df = pd.read_csv("data/seeds.csv")
df.head(5)

C:\Users\priya\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,area,perimeter,compactness,kernel_length,kernel_width,asymmetry_coefficient,groove_length,species
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,0
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,0
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,0
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,0
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,0


In [2]:
df['species'].unique()

array([0, 1, 2], dtype=int64)

In [3]:
X = df.drop("species", axis=1)
y = df["species"]

In [4]:
df.isnull().sum()

area                     0
perimeter                0
compactness              0
kernel_length            0
kernel_width             0
asymmetry_coefficient    0
groove_length            0
species                  0
dtype: int64

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled,
    y,
    test_size=0.2,
    random_state=42
)

In [7]:
import torch

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [8]:
print(X_train_tensor)

tensor([[-1.0395, -1.0203, -0.9078,  ..., -1.1109,  0.9966, -0.2061],
        [ 1.5373,  1.5587,  0.4455,  ...,  1.4633, -0.1488,  1.6927],
        [-0.8501, -0.8591, -0.4284,  ..., -0.7155,  1.1812, -0.7405],
        ...,
        [ 1.3651,  1.3284,  0.8315,  ...,  1.1528, -0.3088,  1.3154],
        [-0.9086, -0.8361, -1.0902,  ..., -0.9703, -1.3595, -0.4692],
        [ 1.5890,  1.4896,  1.1666,  ...,  1.6809,  0.4052,  1.2256]])


#Day2

In [7]:
from sklearn.model_selection import train_test_split
# 80% train+val, 20% test
X_temp, X_test, y_temp, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# From train+val → 80% train, 20% val
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=42
)

In [9]:
#PyTorch Tensors
import torch
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)

X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.long)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.long)


In [11]:
from torch.utils.data import TensorDataset, DataLoader
train_loader = DataLoader(
    TensorDataset(X_train, y_train),
    batch_size=16,
    shuffle=True
)

val_loader = DataLoader(
    TensorDataset(X_val, y_val),
    batch_size=16
)

test_loader = DataLoader(
    TensorDataset(X_test, y_test),
    batch_size=16
)


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
class SeedsDNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(7, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 3)
        )

    def forward(self, x):
        return self.model(x)

In [14]:
model = SeedsDNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
epochs = 50

for epoch in range(epochs):
    model.train()
    train_loss = 0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    val_accuracy = correct / total

    if (epoch + 1) % 10 == 0:
        print(
            f"Epoch [{epoch+1}/{epochs}] "
            f"Train Loss: {train_loss:.4f} "
            f"Val Loss: {val_loss:.4f} "
            f"Val Accuracy: {val_accuracy:.4f}"
        )

Epoch [10/50] Train Loss: 5.2735 Val Loss: 1.9674 Val Accuracy: 0.8824
Epoch [20/50] Train Loss: 2.0700 Val Loss: 0.4588 Val Accuracy: 0.9706
Epoch [30/50] Train Loss: 1.7665 Val Loss: 0.3142 Val Accuracy: 0.9412
Epoch [40/50] Train Loss: 1.2259 Val Loss: 0.3070 Val Accuracy: 0.9412
Epoch [50/50] Train Loss: 1.0508 Val Loss: 0.3345 Val Accuracy: 0.9412


In [17]:
from sklearn.metrics import accuracy_score
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.numpy())
        all_labels.extend(y_batch.numpy())

test_accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Test Accuracy: 92.86%
